## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
import tensorflow as tf
from keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME", "USE_CASE", "INCOME_AMT"], 1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
ORGANIZATION                 4
STATUS                       2
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C6100        1
C4120        1
C2570        1
C2600        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_great_one = class_counts[class_counts > 1]
class_count_great_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C2400        6
C1720        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C1234        2
C1267        2
C3200        2
C1246        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1750].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,1,0,1,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,1,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(['IS_SUCCESSFUL'], axis = 'columns').values
y = application_df_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_layer = len(X_train[0])
hidden_node_layers1 = 90
hidden_node_layers2 = 50
hidden_node_layers3 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers1, input_dim = num_input_layer, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers2, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers3, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                2430      
                                                                 
 dense_1 (Dense)             (None, 50)                4550      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,031
Trainable params: 7,031
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Checkpoint file creation
os.makedirs('checkpoints/', exist_ok = True)
checkpoint_dir = r'checkpoints/Checkpoints.{epoch:02d}.hdf5'

In [14]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

checkpoint = ModelCheckpoint(filepath = checkpoint_dir, monitor = 'accuracy', verbose = 1, save_weights_only = True,
                            mode = 'auto', period = 1)

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 750, callbacks = [checkpoint])

Epoch 1/750
749/804 [==========================>...] - ETA: 0s - loss: 0.5801 - accuracy: 0.7138
Epoch 1: saving model to checkpoints\Checkpoints.01.hdf5
804/804 [==============================] - 1s 686us/step - loss: 0.5806 - accuracy: 0.7132
Epoch 2/750
767/804 [===========================>..] - ETA: 0s - loss: 0.5719 - accuracy: 0.7195
Epoch 2: saving model to checkpoints\Checkpoints.02.hdf5
804/804 [==============================] - 1s 669us/step - loss: 0.5727 - accuracy: 0.7185
Epoch 3/750
774/804 [===========================>..] - ETA: 0s - loss: 0.5703 - accuracy: 0.7181
Epoch 3: saving model to checkpoints\Checkpoints.03.hdf5
804/804 [==============================] - 1s 664us/step - loss: 0.5701 - accuracy: 0.7182
Epoch 4/750
778/804 [============================>.] - ETA: 0s - loss: 0.5688 - accuracy: 0.7198
Epoch 4: saving model to checkpoints\Checkpoints.04.hdf5
804/804 [==============================] - 1s 663us/step - loss: 0.5692 - accuracy: 0.7195
Epoch 5/750
779/804 

780/804 [============================>.] - ETA: 0s - loss: 0.5614 - accuracy: 0.7239
Epoch 34: saving model to checkpoints\Checkpoints.34.hdf5
804/804 [==============================] - 1s 660us/step - loss: 0.5614 - accuracy: 0.7241
Epoch 35/750
772/804 [===========================>..] - ETA: 0s - loss: 0.5610 - accuracy: 0.7253
Epoch 35: saving model to checkpoints\Checkpoints.35.hdf5
804/804 [==============================] - 1s 664us/step - loss: 0.5616 - accuracy: 0.7245
Epoch 36/750
756/804 [===========================>..] - ETA: 0s - loss: 0.5613 - accuracy: 0.7247
Epoch 36: saving model to checkpoints\Checkpoints.36.hdf5
804/804 [==============================] - 1s 681us/step - loss: 0.5613 - accuracy: 0.7247
Epoch 37/750
740/804 [==========================>...] - ETA: 0s - loss: 0.5618 - accuracy: 0.7239
Epoch 37: saving model to checkpoints\Checkpoints.37.hdf5
804/804 [==============================] - 1s 690us/step - loss: 0.5610 - accuracy: 0.7245
Epoch 38/750
776/804 [===

756/804 [===========================>..] - ETA: 0s - loss: 0.5587 - accuracy: 0.7264
Epoch 67: saving model to checkpoints\Checkpoints.67.hdf5
804/804 [==============================] - 1s 679us/step - loss: 0.5587 - accuracy: 0.7266
Epoch 68/750
778/804 [============================>.] - ETA: 0s - loss: 0.5583 - accuracy: 0.7265
Epoch 68: saving model to checkpoints\Checkpoints.68.hdf5
804/804 [==============================] - 1s 656us/step - loss: 0.5590 - accuracy: 0.7264
Epoch 69/750
775/804 [===========================>..] - ETA: 0s - loss: 0.5592 - accuracy: 0.7266
Epoch 69: saving model to checkpoints\Checkpoints.69.hdf5
804/804 [==============================] - 1s 663us/step - loss: 0.5590 - accuracy: 0.7266
Epoch 70/750
780/804 [============================>.] - ETA: 0s - loss: 0.5591 - accuracy: 0.7267
Epoch 70: saving model to checkpoints\Checkpoints.70.hdf5
804/804 [==============================] - 1s 658us/step - loss: 0.5587 - accuracy: 0.7270
Epoch 71/750
781/804 [===

779/804 [============================>.] - ETA: 0s - loss: 0.5592 - accuracy: 0.7266
Epoch 100: saving model to checkpoints\Checkpoints.100.hdf5
804/804 [==============================] - 1s 656us/step - loss: 0.5584 - accuracy: 0.7272
Epoch 101/750
783/804 [============================>.] - ETA: 0s - loss: 0.5579 - accuracy: 0.7266
Epoch 101: saving model to checkpoints\Checkpoints.101.hdf5
804/804 [==============================] - 1s 655us/step - loss: 0.5578 - accuracy: 0.7271
Epoch 102/750
760/804 [===========================>..] - ETA: 0s - loss: 0.5571 - accuracy: 0.7282
Epoch 102: saving model to checkpoints\Checkpoints.102.hdf5
804/804 [==============================] - 1s 674us/step - loss: 0.5577 - accuracy: 0.7273
Epoch 103/750
771/804 [===========================>..] - ETA: 0s - loss: 0.5589 - accuracy: 0.7262
Epoch 103: saving model to checkpoints\Checkpoints.103.hdf5
804/804 [==============================] - 1s 665us/step - loss: 0.5576 - accuracy: 0.7275
Epoch 104/750


Epoch 133/750
778/804 [============================>.] - ETA: 0s - loss: 0.5556 - accuracy: 0.7290
Epoch 133: saving model to checkpoints\Checkpoints.133.hdf5
804/804 [==============================] - 1s 659us/step - loss: 0.5568 - accuracy: 0.7276
Epoch 134/750
781/804 [============================>.] - ETA: 0s - loss: 0.5579 - accuracy: 0.7271
Epoch 134: saving model to checkpoints\Checkpoints.134.hdf5
804/804 [==============================] - 1s 651us/step - loss: 0.5574 - accuracy: 0.7277
Epoch 135/750
784/804 [============================>.] - ETA: 0s - loss: 0.5562 - accuracy: 0.7282
Epoch 135: saving model to checkpoints\Checkpoints.135.hdf5
804/804 [==============================] - 1s 654us/step - loss: 0.5566 - accuracy: 0.7276
Epoch 136/750
779/804 [============================>.] - ETA: 0s - loss: 0.5558 - accuracy: 0.7281
Epoch 136: saving model to checkpoints\Checkpoints.136.hdf5
804/804 [==============================] - 1s 655us/step - loss: 0.5563 - accuracy: 0.7277


...

Epoch 595/750
783/804 [============================>.] - ETA: 0s - loss: 0.5550 - accuracy: 0.7296
Epoch 595: saving model to checkpoints\Checkpoints.595.hdf5
804/804 [==============================] - 1s 655us/step - loss: 0.5552 - accuracy: 0.7294
Epoch 596/750
775/804 [===========================>..] - ETA: 0s - loss: 0.5537 - accuracy: 0.7282
Epoch 596: saving model to checkpoints\Checkpoints.596.hdf5
804/804 [==============================] - 1s 661us/step - loss: 0.5537 - accuracy: 0.7285
Epoch 597/750
778/804 [============================>.] - ETA: 0s - loss: 0.5539 - accuracy: 0.7294
Epoch 597: saving model to checkpoints\Checkpoints.597.hdf5
804/804 [==============================] - 1s 659us/step - loss: 0.5536 - accuracy: 0.7295
Epoch 598/750
779/804 [============================>.] - ETA: 0s - loss: 0.5542 - accuracy: 0.7293
Epoch 598: saving model to checkpoints\Checkpoints.598.hdf5
804/804 [==============================] - 1s 658us/step - loss: 0.5539 - accuracy: 0.7294


Epoch 628/750
786/804 [============================>.] - ETA: 0s - loss: 0.5537 - accuracy: 0.7294
Epoch 628: saving model to checkpoints\Checkpoints.628.hdf5
804/804 [==============================] - 1s 646us/step - loss: 0.5537 - accuracy: 0.7292
Epoch 629/750
787/804 [============================>.] - ETA: 0s - loss: 0.5543 - accuracy: 0.7292
Epoch 629: saving model to checkpoints\Checkpoints.629.hdf5
804/804 [==============================] - 1s 645us/step - loss: 0.5538 - accuracy: 0.7297
Epoch 630/750
784/804 [============================>.] - ETA: 0s - loss: 0.5547 - accuracy: 0.7290
Epoch 630: saving model to checkpoints\Checkpoints.630.hdf5
804/804 [==============================] - 1s 648us/step - loss: 0.5538 - accuracy: 0.7293
Epoch 631/750
784/804 [============================>.] - ETA: 0s - loss: 0.5542 - accuracy: 0.7286
Epoch 631: saving model to checkpoints\Checkpoints.631.hdf5
804/804 [==============================] - 1s 649us/step - loss: 0.5534 - accuracy: 0.7294


Epoch 661/750
779/804 [============================>.] - ETA: 0s - loss: 0.5537 - accuracy: 0.7297
Epoch 661: saving model to checkpoints\Checkpoints.661.hdf5
804/804 [==============================] - 1s 651us/step - loss: 0.5535 - accuracy: 0.7292
Epoch 662/750
802/804 [============================>.] - ETA: 0s - loss: 0.5534 - accuracy: 0.7296
Epoch 662: saving model to checkpoints\Checkpoints.662.hdf5
804/804 [==============================] - 1s 698us/step - loss: 0.5533 - accuracy: 0.7297
Epoch 663/750
759/804 [===========================>..] - ETA: 0s - loss: 0.5531 - accuracy: 0.7295
Epoch 663: saving model to checkpoints\Checkpoints.663.hdf5
804/804 [==============================] - 1s 668us/step - loss: 0.5531 - accuracy: 0.7294
Epoch 664/750
779/804 [============================>.] - ETA: 0s - loss: 0.5534 - accuracy: 0.7294
Epoch 664: saving model to checkpoints\Checkpoints.664.hdf5
804/804 [==============================] - 1s 651us/step - loss: 0.5532 - accuracy: 0.7296


Epoch 694/750
774/804 [===========================>..] - ETA: 0s - loss: 0.5524 - accuracy: 0.7302
Epoch 694: saving model to checkpoints\Checkpoints.694.hdf5
804/804 [==============================] - 1s 655us/step - loss: 0.5532 - accuracy: 0.7297
Epoch 695/750
756/804 [===========================>..] - ETA: 0s - loss: 0.5538 - accuracy: 0.7290
Epoch 695: saving model to checkpoints\Checkpoints.695.hdf5
804/804 [==============================] - 1s 680us/step - loss: 0.5535 - accuracy: 0.7294
Epoch 696/750
743/804 [==========================>...] - ETA: 0s - loss: 0.5554 - accuracy: 0.7272
Epoch 696: saving model to checkpoints\Checkpoints.696.hdf5
804/804 [==============================] - 1s 752us/step - loss: 0.5533 - accuracy: 0.7295
Epoch 697/750
804/804 [==============================] - ETA: 0s - loss: 0.5534 - accuracy: 0.7299
Epoch 697: saving model to checkpoints\Checkpoints.697.hdf5
804/804 [==============================] - 1s 693us/step - loss: 0.5534 - accuracy: 0.7299


Epoch 727/750
773/804 [===========================>..] - ETA: 0s - loss: 0.5527 - accuracy: 0.7305
Epoch 727: saving model to checkpoints\Checkpoints.727.hdf5
804/804 [==============================] - 1s 785us/step - loss: 0.5536 - accuracy: 0.7297
Epoch 728/750
800/804 [============================>.] - ETA: 0s - loss: 0.5536 - accuracy: 0.7301
Epoch 728: saving model to checkpoints\Checkpoints.728.hdf5
804/804 [==============================] - 1s 695us/step - loss: 0.5537 - accuracy: 0.7300
Epoch 729/750
784/804 [============================>.] - ETA: 0s - loss: 0.5531 - accuracy: 0.7301
Epoch 729: saving model to checkpoints\Checkpoints.729.hdf5
804/804 [==============================] - 1s 710us/step - loss: 0.5532 - accuracy: 0.7298
Epoch 730/750
772/804 [===========================>..] - ETA: 0s - loss: 0.5532 - accuracy: 0.7297
Epoch 730: saving model to checkpoints\Checkpoints.730.hdf5
804/804 [==============================] - 1s 656us/step - loss: 0.5532 - accuracy: 0.7294


In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5783 - accuracy: 0.7228 - 202ms/epoch - 754us/step
Loss: 0.5783398151397705, Accuracy: 0.7227988243103027


In [17]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')